In [1]:
from datasets import load_from_disk, load_dataset
from datasets import load_dataset, load_from_disk
import sys
import json
from pycocoevalcap.bleu.bleu import Bleu
# from pycocoevalcap.meteor.meteor import Meteor
from pycocoevalcap.rouge.rouge import Rouge
from pycocoevalcap.cider.cider import Cider
mplug = load_dataset('alexshengzhili/mPLUG-owl', split = '1_percent_as_validation')


/opt/conda/envs/llava/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset parquet (/home/ubuntu/.cache/huggingface/datasets/alexshengzhili___parquet/alexshengzhili--mPLUG-owl-803e207c32dd7d84/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


In [2]:
import json

# load this file /home/ubuntu/LLaVA-Graph/evaluation/llava-7b-v0/llava_answers_3k.jsonl
# into a list of dictionaries
with open('llava_answers_3k.jsonl', 'r') as f:
    llava_answers_3k = [json.loads(line) for line in f]

In [3]:
llava_list  = [item['text'] for item in llava_answers_3k]

In [5]:
for i in range(3000):
    assert llava_answers_3k[i]['prompt'] == mplug[i]['q_a_pairs'][0][0]

In [6]:
mplug.add_column('llava_response', llava_list)

Dataset({
    features: ['image_file', 'id', 'caption', 'conversations', 'first_mention', 'response', 'title', 'abstract', 'q_a_pairs', 'response_mPLUG-owl', 'llava_response'],
    num_rows: 3002
})

In [7]:
llava = mplug.remove_columns(['response_mPLUG-owl'])

In [8]:
llava = llava.add_column('llava_response', llava_list)

In [10]:
llava

Dataset({
    features: ['image_file', 'id', 'caption', 'conversations', 'first_mention', 'response', 'title', 'abstract', 'q_a_pairs', 'llava_response'],
    num_rows: 3002
})

In [11]:
def evaluate_captions(gt_captions, pred_captions, print_scores=True):
    evaluators = [Bleu(4), Rouge(), Cider()]
    
    results = {}
    for evaluator in evaluators:
        score, _ = evaluator.compute_score(gt_captions, pred_captions)
        if isinstance(evaluator, Bleu):
            score = score[-1]  # Use the BLEU-4 score
        results[evaluator.method()] = score
    
    if print_scores:
        print("----- Evaluation Scores -----")
        for metric, score in results.items():
            print(f"{metric}: {score}")
    
    return results

def captions_list_to_dict(captions_list):
    captions_dict = {}
    for idx, caption in enumerate(captions_list):
        captions_dict[idx] = [caption]
    return captions_dict

gt_mplug = [example[0][1] for example in llava['q_a_pairs']]
pred_mplug = [example['llava_response'] for example in llava]
evaluate_captions(captions_list_to_dict(gt_mplug),
                    captions_list_to_dict(pred_mplug))

{'testlen': 209903, 'reflen': 189848, 'guess': [209903, 206901, 204007, 201272], 'correct': [53732, 17554, 8886, 5651]}
ratio: 1.1056371412919752
----- Evaluation Scores -----
Bleu: 0.07178897661920974
Rouge: 0.18558197583627023
CIDEr: 0.047755558241033114


{'Bleu': 0.07178897661920974,
 'Rouge': 0.18558197583627023,
 'CIDEr': 0.047755558241033114}

In [16]:
llava.push_to_hub('alexshengzhili/llava-eval', type = 'dataset')


Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:01<00:00,  1.68s/it]
